In [1]:
import numpy as np
import cv2
import os
import math
import matplotlib.pyplot as plt

In [2]:
def face_detection(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_frontalface_default.xml')

    face = haar_classifier.detectMultiScale(image_gray, scaleFactor=1.3, minNeighbors=7)
    
    if len(face) != 0:
        (x,y,w,h) = face[0]
        
        dim = (100,100)
        img = image_gray[y:y+w, x:x+h]
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        return resized, face[0]
    else:
        return -1

In [3]:
# Apply LBP Algorithm on a block of the image
def get_lbp_hist(grayscale_img):
    
    dim = grayscale_img.shape
    
    lbp = np.zeros(256)
   
    for i in range(1,dim[0]-1):
        for j in range(1,dim[1]-1):
            
            pixel = grayscale_img[i][j]
            neighbours = np.concatenate([grayscale_img[i-1][j-1:j+2] , [ grayscale_img[i][j+1] ] , grayscale_img[i+1][j-1:j+2][::-1],[grayscale_img[i][j-1]]] )
            neighbours[ neighbours <= pixel] = '0'
            neighbours[ neighbours > pixel]  = '1'
            binary = ''.join( str(c) for c in neighbours )
            
            binary = int(binary,2)
            lbp[binary]+=1
            
    return lbp # <LBP histogram which is a list of 256 numbers>




In [4]:
# Segment the image into 7x7 blocks,apply LBP algorithm on them, then Concatente all hitograms into one
def segment_img(img):
    dim = img.shape
    patch_width  = 7
    patch_height = 7
    
    
    histograms = []
    
    for x in range(0,dim[0],patch_width):
        for y in range(0,dim[1],patch_height):
        
            if(patch_width+x >= dim[0] or patch_width+y >= dim[1] ):
                continue
            patch = img[x:x+patch_width,y:y+patch_height]
    
            histogram = get_lbp_hist(patch)
            histograms.append(histogram)
    
    histo = np.concatenate( [ h for h in histograms ] )
    
    return histo

In [5]:
# Training on images for "tom" and "clinton"

classes = []
train_hist  = []
train_labels = []

def train_data():
    
    
    for person in os.listdir('training'):
        
        classes.append(person)
        
        for image in os.listdir('training/'+person):
            img_path = 'training/'+person+'/'+image
            img = cv2.imread(img_path)
            result= face_detection(img)
            if result == -1:
                continue
            else:
                img = result[0]
            histo = segment_img(img)
            train_hist.append(histo)
            train_labels.append(person)
train_data()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'training'

In [0]:
#print(train_labels)
#print ( len(train_hist)  )

In [0]:
# Apply Nearest Neighbour Algorithm for test image 
def classify(img,face=-1):
    
    
    if face == -1:
        img,_ = face_detection(img)
      
    test_hist = segment_img(img)
    
    
    mini_dist  = 1000000
    mini_class = -1
    
    index = np.argmin(np.sum(np.abs(train_hist[:] - test_hist),axis=1))
    
    #if mini_dist > 5500:
    #    return -1
    
    mini_class = train_labels[index]
    return mini_class

In [0]:
def test_img(img,face=-1):

    c = classify(img,face)    
    if c != -1:
        return c
    else:
        return "No Match"
    

In [0]:
cap = cv2.VideoCapture(0) 

while 1: 
    ret, img = cap.read()     
    res = face_detection(img)

    if res != -1:
        img2 = res[0]
        face = res[1]
        (x,y,w,h) = face
        
        c = test_img(img2,1)
        
        img = cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        img = cv2.putText(img, c, (x,y), cv2.FONT_HERSHEY_PLAIN, 2.5, (0, 0, 255), 2)
    
    cv2.imshow('Face Recognition',img) 
    
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break
cap.release() 
cv2.destroyAllWindows() 

In [0]:
#img = cv2.imread('test/16.jpg')
#test_img(img)

In [0]:
#train = np.array([[1,2,3],[4,5,6],[7,8,9]])
#test = np.array([5,5,5])
#res = np.sum(np.abs(train[:] - test),axis=1)
#print(np.argmin(res))
#grayscale_img = [[1,2,3],[4,5,6],[7,8,9]]
#i=1
#j=1
#pixel = grayscale_img[i][j]
            
#neighbours = np.concatenate([grayscale_img[i-1][j-1:j+2] , [ grayscale_img[i][j+1] ] , grayscale_img[i+1][j-1:j+2][::-1],[grayscale_img[i][j-1]] ])
#conc = np.concatenate( [grayscale_img[i-1][j-1:j+2], [grayscale_img[i][j+1]]] )

#print(neighbours)
#neighbours[ neighbours <= pixel] = str(0)
#neighbours[ neighbours > pixel] = str(1)
#binary = ''.join( str(c) for c in neighbours )
#binary = int(binary,2)
#binary

#arr = [np.array([1,2,3]),np.array([4,5,6]),np.array([7,8,9]) ]

#conc = np.concatenate([ x for x in arr ] )

#print(conc)